<a href="https://colab.research.google.com/github/TanayPhatak/Google_Colab_Projects/blob/main/nanoGPT_code_along.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://www.youtube.com/watch?v=kCc8FmEb1nY

# Dev section

In [ ]:
# Import dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare//input.txt

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [ ]:
# Hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [ ]:
# read the data file
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print('Length of dataset in characters: ', len(text))

In [ ]:
print(text[:1000])

In [ ]:
chars = sorted(list(set(text))) # sorted list of unique characters in the txt
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encode: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: take a list of integers, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

In [ ]:
# encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
temp = decode(data.tolist())
print(temp[:1000])

In [ ]:
# Split data into train and validation datasets
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [ ]:
# block_size = 8
# train_data[:block_size+1]

In [ ]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#   context = x[:t+1]
#   target = y[t]
#   print(f"When input is {context.tolist()} - the target is: {target}")

In [ ]:
# batch_size = 4 # how many independent sequences will we process in parallel?
# block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
  # generate a small batch of data of imputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  # x, y = x.to(device), y.to(device)
  return x, y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)
print('Targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'When input is {context.tolist()} - target is: {target.tolist()}')


In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = m(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
    m.train()
    return out

## Bigram model

In [ ]:
# Bigram model

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logit for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are both (B, T) tensor of integers
    logits = self.token_embedding_table(idx) # (B, T, C) - B:batch_size, T (time):block_size, C (channel):vocab_size

    if targets == None:
      loss = None
    else:
      # pytorch cross_entropy module requires input of shape (minibatch, C) for logits, (minibatch) for targets
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index tothe running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
# m = model.to(device)

logits, loss = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

In [ ]:
# Create PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"Step {iter}: train loss - {losses['train']:.4f}, val loss - {losses}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

In [ ]:
# batch_size = 32
# for steps in range(10000):
#   # sample a batch of data
#   xb, yb = get_batch('train')

#   # evaluate the loss
#   logits, loss = m(xb, yb)
#   optimizer.zero_grad(set_to_none=True)
#   loss.backward()
#   optimizer.step()

# print(loss.item())

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

## The mathematical trick in self-attention

In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channel
x = torch.randn(B, T, C)
x.shape

In [ ]:
# We want x[b, t] = mean_{i <= t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # (t, C)
    xbow[b, t] = torch.mean(xprev, 0)

# Not efficient

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ---> (B, T, C)
                # (T, T) broadcasted to (B, T, T)
torch.allclose(xbow, xbow2)

# One of the efficient ways

In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

# Another efficient way using softmax

In [ ]:
xbow[0]

In [ ]:
xbow2[0]

In [ ]:
xbow3[0]

In [ ]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

In [ ]:
wei[0]

---
---
---

# Main flow

In [ ]:
# Import dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare//input.txt

In [ ]:
import time
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# Hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)

In [ ]:
# read the data file
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
# all the unique characters that occur in the txt
chars = sorted(list(set(text))) # sorted list of unique characters in the txt
vocab_size = len(chars)

In [ ]:
# Create a mapping from the characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encode: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: take a list of integers, output a string

In [ ]:
# Split data into train and validation datasets
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [ ]:
def get_batch(split):
  # generate a small batch of data of imputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x) # (B, T, C)
    q = self.query(x) # (B, T, C)
    # compute attention scores ('affinitiess')
    wei = q @ k.transpose(-2, -1) * C**-5 # (B, T, C) @ (B, C, T) --> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  ''' multiple heads of self-attention in parallel '''

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.head], dim=-1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):
  ''' a simple linear layer followed bya a non-linearity '''

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )


  def forward(self, x):
    return self.net(x)

In [ ]:
class Block(nn.Module):
  ''' Transformer block: communication followed by computation '''

  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: no. of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [ ]:
# Simple Bigram model

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    # each token directly reads off the logit for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)

    # self.blocks = nn.Sequential(
    #     Block(n_embd, n_head=4),
    #     Block(n_embd, n_head=4),
    #     Block(n_embd, n_head=4),
    #     nn.LayerNorm(n_embd)
    # )
    # self.sa_head = Head(n_embd)
    # self.sa_heads = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
    # self.ffwd = FeedForward(n_embd)
    # self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are both (B, T) tensor of integers
    tok_emb = self.token_embedding_table(idx) # (B, T, C) - B:batch_size, T (time):block_size, C (channel):vocab_size
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = tok_emb + pos_emb # (B, T, C)
    # x = self.sa_head(x) # apply one head of self-attention (B, T, C)
    x = self.blocks(x)
    x = self.ln_f(x) # (B, T, C)
    # x = self.sa_heads(x)
    # x = self.ffwd(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets == None:
      loss = None
    else:
      # pytorch cross_entropy module requires input of shape (minibatch, C) for logits, (minibatch) for targets
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index tothe running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [ ]:
'''Model Training'''

# Model initialization
model = BigramLanguageModel()
m = model.to(device)

# Create PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
s = time.time()
# training loop
for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    e = time.time()
    print(f"Step {iter}: train loss - {losses['train']:.4f}, val loss - {losses['val']:.4f}, time - {e-s} seconds")
    s = time.time()
  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))